In [1]:
!pip install opencv-python face_recognition pandas numpy matplotlib scipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=1599c174e9efd2dca8749048062cf0234fcc80d1463ad811e53b69d9c9874b2b
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cv2
import face_recognition
df=pd.read_csv('/content/Assignment Data - Sheet1.csv')
df.head()

,Performance,Video URL
0,1.1060,https://fgimagestorage.blob.core.windows.net/f...
1,2.2447,https://fgimagestorage.blob.core.windows.net/f...
2,2.0126,https://fgimagestorage.blob.core.windows.net/f...
3,1.7708,https://fgimagestorage.blob.core.windows.net/f...
4,0.6293,https://fgimagestorage.blob.core.windows.net/f...


In [2]:
df=df.iloc[173:183]

In [3]:
df.reset_index( inplace=True)

In [4]:
df.reset_index( drop=True)

,index,Performance,Video URL
0,173,0.111732,https://fgimagestorage.blob.core.windows.net/f...
1,174,1.063129,https://fgimagestorage.blob.core.windows.net/f...
2,175,0.120503,https://fgimagestorage.blob.core.windows.net/f...
3,176,0.026820,https://fgimagestorage.blob.core.windows.net/f...
4,177,0.840288,https://fgimagestorage.blob.core.windows.net/f...
5,178,1.796113,https://fgimagestorage.blob.core.windows.net/f...
6,179,0.379376,https://fgimagestorage.blob.core.windows.net/f...
7,180,0.024163,https://fgimagestorage.blob.core.windows.net/f...
8,181,0.196793,https://fgimagestorage.blob.core.windows.net/f...
9,182,0.284253,https://fgimagestorage.blob.core.windows.net/f...


In [5]:
df.drop(columns=['index'], inplace=True)

In [6]:
df

,Performance,Video URL
0,0.111732,https://fgimagestorage.blob.core.windows.net/f...
1,1.063129,https://fgimagestorage.blob.core.windows.net/f...
2,0.120503,https://fgimagestorage.blob.core.windows.net/f...
3,0.026820,https://fgimagestorage.blob.core.windows.net/f...
4,0.840288,https://fgimagestorage.blob.core.windows.net/f...
5,1.796113,https://fgimagestorage.blob.core.windows.net/f...
6,0.379376,https://fgimagestorage.blob.core.windows.net/f...
7,0.024163,https://fgimagestorage.blob.core.windows.net/f...
8,0.196793,https://fgimagestorage.blob.core.windows.net/f...
9,0.284253,https://fgimagestorage.blob.core.windows.net/f...


In [7]:
!pip install opencv-python dlib face-recognition


In [8]:
df

,Performance,Video URL
0,0.111732,https://fgimagestorage.blob.core.windows.net/f...
1,1.063129,https://fgimagestorage.blob.core.windows.net/f...
2,0.120503,https://fgimagestorage.blob.core.windows.net/f...
3,0.026820,https://fgimagestorage.blob.core.windows.net/f...
4,0.840288,https://fgimagestorage.blob.core.windows.net/f...
5,1.796113,https://fgimagestorage.blob.core.windows.net/f...
6,0.379376,https://fgimagestorage.blob.core.windows.net/f...
7,0.024163,https://fgimagestorage.blob.core.windows.net/f...
8,0.196793,https://fgimagestorage.blob.core.windows.net/f...
9,0.284253,https://fgimagestorage.blob.core.windows.net/f...


In [9]:
import os
import cv2
import face_recognition
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Directory paths
videos_folder = "videos"  # Folder containing input videos
embeddings_folder = "embeddings"  # Folder to store embeddings

# Ensure embeddings folder exists
os.makedirs(embeddings_folder, exist_ok=True)

# Step 1: Extract embeddings and save them
def extract_and_save_embeddings(videos_folder, embeddings_folder):
    for video_file in os.listdir(videos_folder):
        if video_file.endswith((".mp4", ".avi", ".mov", ".mkv")):
            video_path = os.path.join(videos_folder, video_file)
            video_name = os.path.splitext(video_file)[0]  # Get the filename without extension
            embedding_path = os.path.join(embeddings_folder, f"{video_name}.npy")

            print(f"Processing video: {video_file}")
            # Extract the most prominent face embedding
            embedding = extract_most_prominent_face_embedding(video_path)

            if embedding is not None:
                np.save(embedding_path, embedding)
                print(f"Embedding saved: {embedding_path}")
            else:
                print(f"No face detected in video: {video_file}")

# Function to extract the most prominent face embedding from a video
def extract_most_prominent_face_embedding(video_path):
    video_capture = cv2.VideoCapture(video_path)
    highest_confidence = 0
    most_prominent_face_embedding = None

    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame, model="cnn")
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for face_location, face_encoding in zip(face_locations, face_encodings):
            top, right, bottom, left = face_location
            confidence_score = (bottom - top) * (right - left)

            if confidence_score > highest_confidence:
                highest_confidence = confidence_score
                most_prominent_face_embedding = face_encoding

    video_capture.release()
    return most_prominent_face_embedding

# Step 2: Function to calculate similarity between two embeddings
def calculate_similarity(embedding_file1, embedding_file2):
    # Load embeddings
    embedding1 = np.load(embedding_file1)
    embedding2 = np.load(embedding_file2)

    # Reshape for compatibility
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    # Calculate cosine similarity
    similarity_score = cosine_similarity(embedding1, embedding2)[0][0]
    return similarity_score




In [10]:
df.head()

,Performance,Video URL
0,0.111732,https://fgimagestorage.blob.core.windows.net/f...
1,1.063129,https://fgimagestorage.blob.core.windows.net/f...
2,0.120503,https://fgimagestorage.blob.core.windows.net/f...
3,0.026820,https://fgimagestorage.blob.core.windows.net/f...
4,0.840288,https://fgimagestorage.blob.core.windows.net/f...


In [11]:
import pandas as pd
import requests
import os

# Path to the CSV file
# csv_file_path = "dataset.csv"  # Replace with the path to your CSV file
output_folder = "videos"  # Folder to save the downloaded videos

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Read the dataset
# df = pd.read_csv(csv_file_path)

# Function to download a video
def download_video(video_url, save_path):
    try:
        response = requests.get(video_url, stream=True)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Save the video in chunks
        with open(save_path, "wb") as video_file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:  # Filter out keep-alive chunks
                    video_file.write(chunk)
        print(f"Downloaded: {save_path}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {video_url}: {e}")

# Iterate through the dataset and download videos
for index, row in df.iterrows():
    performance = row["Performance"]
    video_url = row["Video URL"]

    # Generate a unique filename based on the row index and performance value
    filename = f"video_{index}_perf_{performance:.6f}.mp4"
    save_path = os.path.join(output_folder, filename)

    # Download the video
    download_video(video_url, save_path)


Downloaded: videos/video_0_perf_0.111732.mp4
Downloaded: videos/video_1_perf_1.063129.mp4
Downloaded: videos/video_2_perf_0.120503.mp4
Downloaded: videos/video_3_perf_0.026820.mp4
Downloaded: videos/video_4_perf_0.840288.mp4
Downloaded: videos/video_5_perf_1.796113.mp4
Downloaded: videos/video_6_perf_0.379376.mp4
Downloaded: videos/video_7_perf_0.024163.mp4
Downloaded: videos/video_8_perf_0.196793.mp4
Downloaded: videos/video_9_perf_0.284253.mp4


In [12]:
# Run Step 1
os.makedirs(embeddings_folder, exist_ok=True)

extract_and_save_embeddings(videos_folder, embeddings_folder)



Processing video: video_9_perf_0.284253.mp4
Embedding saved: embeddings/video_9_perf_0.284253.npy
Processing video: video_6_perf_0.379376.mp4
Embedding saved: embeddings/video_6_perf_0.379376.npy
Processing video: video_4_perf_0.840288.mp4
Embedding saved: embeddings/video_4_perf_0.840288.npy
Processing video: video_0_perf_0.111732.mp4
Embedding saved: embeddings/video_0_perf_0.111732.npy
Processing video: video_7_perf_0.024163.mp4
Embedding saved: embeddings/video_7_perf_0.024163.npy
Processing video: video_2_perf_0.120503.mp4
Embedding saved: embeddings/video_2_perf_0.120503.npy
Processing video: video_8_perf_0.196793.mp4
Embedding saved: embeddings/video_8_perf_0.196793.npy
Processing video: video_1_perf_1.063129.mp4
No face detected in video: video_1_perf_1.063129.mp4
Processing video: video_5_perf_1.796113.mp4
Embedding saved: embeddings/video_5_perf_1.796113.npy
Processing video: video_3_perf_0.026820.mp4
Embedding saved: embeddings/video_3_perf_0.026820.npy


In [13]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Directory containing embeddings
embeddings_folder = "embeddings"  # Replace with the path to your embeddings folder
output_csv = "similar_pairs_with_avg_performance.csv"

# Function to calculate similarity score between two embeddings
def calculate_similarity(embedding1, embedding2):
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)
    return cosine_similarity(embedding1, embedding2)[0][0]

# List all embeddings
embedding_files = [f for f in os.listdir(embeddings_folder) if f.endswith(".npy")]

# Parse video indices and performance scores from filenames
video_data = []
for embedding_file in embedding_files:
    # Example filename: video_0_perf_0.111732.npy
    parts = embedding_file.replace(".npy", "").split("_")
    video_index = parts[1]
    performance_score = float(parts[3])
    video_data.append({"file": embedding_file, "index": video_index, "performance": performance_score})

# Convert to DataFrame for easier processing
video_df = pd.DataFrame(video_data)

# DataFrame to store results
result_data = []

# Compare all pairs of embeddings
for i in range(len(video_df)):
    for j in range(i + 1, len(video_df)):
        video1 = video_df.iloc[i]
        video2 = video_df.iloc[j]
        # Load embeddings
        embedding1 = np.load(os.path.join(embeddings_folder, video1["file"]))
        embedding2 = np.load(os.path.join(embeddings_folder, video2["file"]))
        embedding1=embedding1 / np.linalg.norm(embedding1)
        embedding2=embedding2 / np.linalg.norm(embedding2)
        # Calculate similarity
        similarity_score = calculate_similarity(embedding1, embedding2)

        # Check if similarity is above 95%
        if similarity_score > 0.85:
            avg_performance = (video1["performance"] + video2["performance"]) / 2
            result_data.append({
                "video_pair": f"video({video1['index']}),video({video2['index']})",
                "average_performance": avg_performance
            })

# Create result DataFrame
result_df = pd.DataFrame(result_data)
# Save to CSV
result_df.to_csv(output_csv, index=False)
print(f"Results saved to {output_csv}")


Results saved to similar_pairs_with_avg_performance.csv


In [16]:
import pandas as pd

# Input DataFrame
df=result_df

# Function to extract the primary video index (e.g., video(3) from "video(3),video(8)")
def extract_primary_video(video_pair):
    return video_pair.split(",")[0]  # Extract the first video in the pair

# Add a column for the primary video
df["primary_video"] = df["video_pair"].apply(extract_primary_video)

# Group by primary video and compute the combined average performance score
grouped_df = (
    df.groupby("primary_video")
    .agg({
        "video_pair": lambda x: ", ".join(x),  # Combine all related video pairs into a single string
        "average_performance": "mean"  # Compute the mean of performance scores
    })
    .reset_index()
)

# Rename columns for clarity
grouped_df.rename(columns={
    "primary_video": "primary_video_group",
    "video_pair": "merged_video_pairs",
    "average_performance": "combined_average_performance"
}, inplace=True)
grouped_df = grouped_df.sort_values(by="combined_average_performance", ascending=False)
# Save or display the results
print(grouped_df)

# Optionally, save the results to a CSV file
grouped_df.to_csv("grouped_results.csv", index=False)


  primary_video_group                                 merged_video_pairs  \
3            video(5)               video(5),video(9), video(5),video(4)   
0            video(0)               video(0),video(5), video(0),video(4)   
7            video(9)                                  video(9),video(4)   
4            video(6)  video(6),video(2), video(6),video(7), video(6)...   
5            video(7)  video(7),video(0), video(7),video(5), video(7)...   
6            video(8)  video(8),video(6), video(8),video(2), video(8)...   
1            video(2)  video(2),video(7), video(2),video(0), video(2)...   
2            video(3)  video(3),video(8), video(3),video(6), video(3)...   

   combined_average_performance  
3                      1.179192  
0                      0.714966  
7                      0.562271  
4                      0.454442  
5                      0.391130  
6                      0.385454  
1                      0.365906  
2                      0.273516  


In [ ]:
#@title trial code
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML
os.makedirs(embeddings_folder, exist_ok=True)

# Directory paths
embeddings_folder = "embeddings"
thumbnails_folder = "thumbnails"  # Folder to store face thumbnails
output_csv = "similar_pairs_with_avg_performance.csv"

# Ensure thumbnails folder exists
os.makedirs(thumbnails_folder, exist_ok=True)

# Function to calculate similarity score
def calculate_similarity(embedding1, embedding2):
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)
    return cosine_similarity(embedding1, embedding2)[0][0]

# Function to extract and save the most prominent face as a thumbnail
def extract_and_save_thumbnail(video_path, save_path):
    video_capture = cv2.VideoCapture(video_path)
    highest_confidence = 0
    best_face = None

    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame, model="cnn")

        for face_location in face_locations:
            top, right, bottom, left = face_location
            confidence_score = (bottom - top) * (right - left)

            if confidence_score > highest_confidence:
                highest_confidence = confidence_score
                best_face = frame[top:bottom, left:right]

    video_capture.release()

    if best_face is not None:
        # Save the thumbnail
        cv2.imwrite(save_path, best_face)
        print(f"Thumbnail saved: {save_path}")
        return save_path
    else:
        print(f"No face found in video: {video_path}")
        return None

# Step 1: Extract thumbnails
video_files = [f for f in os.listdir(embeddings_folder) if f.endswith(".npy")]
video_data = []

for file in video_files:
    parts = file.replace(".npy", "").split("_")
    video_index = parts[1]
    performance_score = float(parts[3])
    video_path = os.path.join("videos", f"video_{video_index}.mp4")  # Assuming videos are in the "videos" folder
    thumbnail_path = os.path.join(thumbnails_folder, f"thumbnail_{video_index}.jpg")

    if not os.path.exists(thumbnail_path):
        thumbnail_path = extract_and_save_thumbnail(video_path, thumbnail_path)

    video_data.append({"index": video_index, "performance": performance_score, "thumbnail": thumbnail_path})

video_df = pd.DataFrame(video_data)

# Step 2: Calculate similarity between embeddings and create results
result_data = []

for i in range(len(video_df)):
    for j in range(i + 1, len(video_df)):
        video1 = video_df.iloc[i]
        video2 = video_df.iloc[j]

        embedding1 = np.load(os.path.join(embeddings_folder, f"video_{video1['index']}_perf_{video1['performance']:.6f}.npy"))
        embedding2 = np.load(os.path.join(embeddings_folder, f"video_{video2['index']}_perf_{video2['performance']:.6f}.npy"))

        similarity_score = calculate_similarity(embedding1, embedding2)

        if similarity_score > 0.95:
            avg_performance = (video1["performance"] + video2["performance"]) / 2
            result_data.append({
                "thumbnail1": video1["thumbnail"],
                "thumbnail2": video2["thumbnail"],
                "average_performance": avg_performance,
                "similarity_score": similarity_score
            })

result_df = pd.DataFrame(result_data)

# Save results to CSV
result_df.to_csv(output_csv, index=False)
print(f"Results saved to {output_csv}")

# Step 3: Display results as HTML
def create_html_with_thumbnails(result_df):
    rows = []
    for _, row in result_df.iterrows():
        thumbnail1_html = f'<img src="{row["thumbnail1"]}" width="100">'
        thumbnail2_html = f'<img src="{row["thumbnail2"]}" width="100">'
        row_html = f"""
        <tr>
            <td>{thumbnail1_html}</td>
            <td>{thumbnail2_html}</td>
            <td>{row["average_performance"]:.6f}</td>
            <td>{row["similarity_score"]:.6f}</td>
        </tr>
        """
        rows.append(row_html)

    html_content = f"""
    <table border="1" style="border-collapse: collapse;">
        <thead>
            <tr>
                <th>Face 1</th>
                <th>Face 2</th>
                <th>Average Performance</th>
                <th>Similarity Score</th>
            </tr>
        </thead>
        <tbody>
            {''.join(rows)}
        </tbody>
    </table>
    """
    return html_content

# Generate and display the HTML
html_content = create_html_with_thumbnails(result_df)
display(HTML(html_content))


No face found in video: videos/video_2.mp4
No face found in video: videos/video_0.mp4
Results saved to similar_pairs_with_avg_performance.csv


Face 1,Face 2,Average Performance,Similarity Score
,,0.116117,0.865872


In [15]:
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd

# Load embeddings and normalize
embeddings_folder = "embeddings"
embedding_files = [f for f in os.listdir(embeddings_folder) if f.endswith(".npy")]

embeddings = []
performance_scores = []
indices = []

for file in embedding_files:
    parts = file.replace(".npy", "").split("_")
    indices.append(parts[1])
    performance_scores.append(float(parts[3]))
    embeddings.append(np.load(os.path.join(embeddings_folder, file)))

# Normalize embeddings
embeddings = [e / np.linalg.norm(e) for e in embeddings]

# Cluster embeddings using DBSCAN
clustering = DBSCAN(eps=0.5, min_samples=2, metric='cosine').fit(embeddings)

# Compute average performance for each cluster
clusters = {}
for idx, cluster_label in enumerate(clustering.labels_):
    if cluster_label == -1:  # Skip noise points
        continue
    if cluster_label not in clusters:
        clusters[cluster_label] = []
    clusters[cluster_label].append(performance_scores[idx])

# Generate DataFrame with results
results = []
for cluster, scores in clusters.items():
    avg_performance = np.mean(scores)
    results.append({"Cluster": cluster, "Average Performance": avg_performance})

results_df = pd.DataFrame(results)
print(results_df)


   Cluster  Average Performance
0        0             0.420005


In [15]:
!pip install deepface tensorflow pandas numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=7391ed9e5d368a0ae1b6e723adbabffe796e64039b23d8c9de31bbf6dc01ea3e
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
